In [7]:

"""
This is an upgraded version of Ceshine's LGBM starter script, simply adding
more average features and weekly average features on it.

Run 1 Store validation  : .py 1s 045
Run 1 Store submission  : .py 1ss 045
Run all Store validation: .py val 045
Run all Store submission: .py a 045


"""
from datetime import date

import pandas as pd
import numpy as np
import lightgbm as lgb
import sys

import gc


from nwrmsle_eval import eval_test

# import math
# import sklearn.metrics as skl_metrics
# from sklearn.metrics import mean_squared_error

from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger

logger = getLogger(__name__)

pd.options.mode.chained_assignment = None  # default='warn'

DIR = '../logs/'

log_fmt = Formatter('%(asctime)s %(name)s %(lineno)d [%(levelname)s]\
    [%(funcName)s] %(message)s ')
handler = StreamHandler()
handler.setLevel('INFO')
handler.setFormatter(log_fmt)
logger.addHandler(handler)

handler = FileHandler(DIR + 'train.py.log', 'a')
handler.setLevel(DEBUG)
handler.setFormatter(log_fmt)
logger.setLevel(DEBUG)
logger.addHandler(handler)

logger.info('start')

##########################################################################
# 1s : validate one store data
# 1ss: submit one store
# val: validate all data
# No parameter: submit all data


param_1 = "1s"
param_2 = "999"


print("input parameter = ", param_1)
print("Test/val number = ", param_2)
submit_filename = '../submit/T' + param_2 + '.csv.gz'
val_filename = '../data/V' + param_2 + '.p'

logger.info(submit_filename)
logger.info(val_filename)

if ((param_1 == "1ss") or (param_1 == "1s")):
    train_out = pd.read_pickle('../data/storeitem_train_1s.p')
    val_out = pd.read_pickle('../data/storeitem_val_1s.p')
    X_test_out = pd.read_pickle('../data/storeitem_test_1s.p')

    item_train_out = pd.read_pickle('../data/item_train_1s.p')
    item_val_out = pd.read_pickle('../data/item_val_1s.p')
    item_X_test_out = pd.read_pickle('../data/item_test_1s.p')

    store_train_out = pd.read_pickle('../data/store_train_1s.p')
    store_val_out = pd.read_pickle('../data/store_val_1s.p')
    store_X_test_out = pd.read_pickle('../data/store_test_1s.p')

    s_f_train_out = pd.read_pickle('../data/storefamily_train_1s.p')
    s_f_val_out = pd.read_pickle('../data/storefamily_val_1s.p')
    s_f_X_test_out = pd.read_pickle('../data/storefamily_test_1s.p')

    df_test = pd.read_csv(
        "../input/test_1s.csv", usecols=[0, 1, 2, 3, 4],
        dtype={'onpromotion': bool},
        parse_dates=["date"]  # , date_parser=parser
    ).set_index(
        ['store_nbr', 'item_nbr', 'date']
    )

else:
    train_out = pd.read_pickle('../data/storeitem_train.p')
    val_out = pd.read_pickle('../data/storeitem_val.p')
    X_test_out = pd.read_pickle('../data/storeitem_test.p')

    item_train_out = pd.read_pickle('../data/item_train.p')
    item_val_out = pd.read_pickle('../data/item_val.p')
    item_X_test_out = pd.read_pickle('../data/item_test.p')

    store_train_out = pd.read_pickle('../data/store_train.p')
    store_val_out = pd.read_pickle('../data/store_val.p')
    store_X_test_out = pd.read_pickle('../data/store_test.p')

    s_f_train_out = pd.read_pickle('../data/storefamily_train.p')
    s_f_val_out = pd.read_pickle('../data/storefamily_val.p')
    s_f_X_test_out = pd.read_pickle('../data/storefamily_test.p')

    df_test = pd.read_csv(
        "../input/test.csv", usecols=[0, 1, 2, 3, 4],
        dtype={'onpromotion': bool},
        parse_dates=["date"]  # , date_parser=parser
    ).set_index(
        ['store_nbr', 'item_nbr', 'date']
    )


# On validation step, need remove last 2 weeks in the train data
if ((param_1 == "val") or (param_1 == "1s")):
    train_out = train_out.loc[train_out["date"] < date(2017, 7, 19), ]

items = pd.read_csv("../input/items.csv",)

items_val = pd.read_csv("../input/items.csv",).set_index("item_nbr")
items_val = items_val.reindex(val_out['item_nbr'])

logger.info('Load data successful')

###############################################################################
# Delete index columns before merge
del train_out["index"]



2018-01-14 10:15:30,871 __main__ 49 [INFO]    [<module>] start 
2018-01-14 10:15:30,871 __main__ 49 [INFO]    [<module>] start 
2018-01-14 10:15:30,871 __main__ 49 [INFO]    [<module>] start 
2018-01-14 10:15:30,871 __main__ 49 [INFO]    [<module>] start 
2018-01-14 10:15:30,878 __main__ 67 [INFO]    [<module>] ../submit/T999.csv.gz 
2018-01-14 10:15:30,878 __main__ 67 [INFO]    [<module>] ../submit/T999.csv.gz 
2018-01-14 10:15:30,878 __main__ 67 [INFO]    [<module>] ../submit/T999.csv.gz 
2018-01-14 10:15:30,878 __main__ 67 [INFO]    [<module>] ../submit/T999.csv.gz 
2018-01-14 10:15:30,891 __main__ 68 [INFO]    [<module>] ../data/V999.p 
2018-01-14 10:15:30,891 __main__ 68 [INFO]    [<module>] ../data/V999.p 
2018-01-14 10:15:30,891 __main__ 68 [INFO]    [<module>] ../data/V999.p 
2018-01-14 10:15:30,891 __main__ 68 [INFO]    [<module>] ../data/V999.p 


input parameter =  1s
Test/val number =  999


2018-01-14 10:15:31,149 __main__ 130 [INFO]    [<module>] Load data successful 
2018-01-14 10:15:31,149 __main__ 130 [INFO]    [<module>] Load data successful 
2018-01-14 10:15:31,149 __main__ 130 [INFO]    [<module>] Load data successful 
2018-01-14 10:15:31,149 __main__ 130 [INFO]    [<module>] Load data successful 


In [8]:
X_train_allF.head(2)

,day_1,mean_14,mean_140,mean_182,mean_21,mean_3,mean_30,mean_364,mean_42,mean_60,...,____ratio_dow_4_2_mean,____ratio_dow_8_2_mean,____ratio_dow_4_3_mean,____ratio_dow_8_3_mean,____ratio_dow_4_4_mean,____ratio_dow_8_4_mean,____ratio_dow_4_5_mean,____ratio_dow_8_5_mean,____ratio_dow_4_6_mean,____ratio_dow_8_6_mean
0,0.0,0.346574,0.047456,0.036504,0.316371,0.231049,0.22146,0.179006,0.158185,0.11073,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
